# Geometry Constrained Feature Matching

In [35]:
import numpy as np 
import random
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import os
import sys
import itertools as it

In [23]:
d = np.load('Ftrue.npy', allow_pickle=True).item()  # Dicctionary
qs= np.load('qs.npy', allow_pickle=True).item()

In [24]:
qs.keys()

dict_keys(['q1', 'q2'])

In [43]:
q1 = qs['q1'] # 2d points in homogenous form 3x8
q2 = qs['q2'] # 2d points in homogenous form 3x8

9

In [5]:
im1 = d['im1']
im2 = d['im2']
R1 = d['R1']
R2 = d['R2']
t1 = np.squeeze(d['t1'] )
t2 = np.squeeze(d['t2'])
K = d['K']

In [14]:
def crossOp(v):
    """
    v: is a vector, should it be in column vector 
    """
    crossv = np.matrix(np.array([[0, -v[2], v[1]],[v[2], 0, -v[0]], [-v[1], v[0], 0]]))
    return crossv

In [15]:
def fundmat(K1, K2, t1, R1):
    """
    K1, K2: Camera Matrices, 
    t1: translation vector camera 1
    R1: Rotation matrix camera 1
    Returns: Fundamental matrix 
    """
    # check the shape of the matrices
    if t1.shape == (3,1):
        pass
    else:
        t1 = t1.T
    
    # Compute the inverses
    K1_inv, K2_inv = np.linalg.inv(K1), np.linalg.inv(K2)
    # Compute essential matrix 
    E = crossOp(t1)@R1
    F = K2_inv.T @ E @ K1_inv
    return F

In [18]:
F = fundmat(K,K,t2,R2)
F

matrix([[ 6.67972386e-12, -7.85049967e-10,  1.17921973e-07],
        [-9.75936980e-10, -4.86806510e-12,  3.28699196e-05],
        [ 4.23506610e-07, -3.21704080e-05, -2.12002228e-04]])

In [42]:
np.kron([1, 10, 100], [5, 6, 4])

array([  5,   6,   4,  50,  60,  40, 500, 600, 400])

In [59]:
q1.shape, q1[:,0]

((3, 8), array([192.80308677, 482.655306  ,   1.        ]))

In [61]:
np.kron(q1[:,0],q2[:,0]).shape

(9,)

In [68]:
def estF_8point(q1, q2)->np.array:
    """
    Given two sets of points in 2D in homogenous form (3,1) from two different images taken with the same camera, 
    we estimate the Fundamental Matrix of the camera.
    
    It applies kronoker product two the pair of points to create the B matrix
    PARAMETERS:
    q1 --> Collection of 2d points on homogenous form (3,n)
    q2 --> Collection of 2d points on homogenous form (3,n)
    
    RETURNS:
    F --> Estimation of the fundamental matrix. 
    """
    # check if we have the same amount of points
    if q1.shape != q2.shape:
        print('q1 and q2 should store the same amount of points')
        return False 
    elif q1.shape[0] != 3:
        print('Make sure your collection of points is in homogenous form')
        return False
    
    # Create the B matrix with the Kronecker product, 
        # Initialize it for faster computation:
    n = q1.shape[0]*q2.shape[0]
    B = np.zeros((q1.shape[1], n))
    # Loop trhoug each point and apply kronoker product to the pair of points
    for i in range(q1.shape[1]):
        B[i,:] = np.kron(q1[:,i],q2[:,i])
        
    # Apply singular value decomposition to obtain 
    _,s,eigVec = np.linalg.svd(B)
    # Find Min 
    idx = np.where(s==min(s))
    # Select the eigenvector correspondent to the eigenvalue
    F_vec = eigVec[idx]
    
    # Reshape to 3x3 
    
#   put option to scale it or not 

    
    return Q.reshape()

In [69]:
estF_8point(q1, q2).shape

(1, 9)

In [20]:
d['q1']

KeyError: 'q1'